In [1]:
# Homework 2 Part 2 (due 7/07/2024)

# Health-care assistance via probabilistic graphical modeling

### Objective
In this project, you will create a health-care assistance bot that can suggest diagnoses for a set of symptoms based on a probabilistic graphical model.

### Step 1: Review 
Review the code from the Bayesian networks exercise.

### Step 2: Acquire data
View this [research article](https://www.nature.com/articles/ncomms5212) and download its supplementary data sets 1, 2 and 3. These data sets include the occurrences of diseases, symptoms, and their co-occurrences in the scientific literature. (For the purpose of this exercise, we are going to assume that the frequency of co-occurrences of diseases and symptoms in scientific papers is proportional to the co-occurence frequencies of actual disease cases and symptoms.)

### Step 3: Create a Bayesian network
Using commands from the `pgmpy` library, create a Bayesian network in which the probability of exhibiting a symptom is conditional on the probability of having an associated disease. 

### Step 4: Initialize priors
Use the disease occurrence data to assign prior probabilities for diseases.

### Step 5: Calculate conditional probability tables
Use the co-occurrence data to define CPTs for each connected pair of disease and symptoms. (Hint: You may need to assign some occurrences of symptoms to an "idiopathic disease" to create valid CPTs.)

### Step 6:
Create a minimal interface in which your bot asks a users for a list of observed symptoms and then returns the name of the disease that is the most likely match to the symptoms. (Hint: Review the input/output commands that you have used in last week's homework.)

In [3]:
import numpy as np
import pandas as pd

# Set Up Data + Tables

In [5]:
diseases = open("41467_2014_BFncomms5212_MOESM1043_ESM.txt", "r")
symptoms = open("41467_2014_BFncomms5212_MOESM1044_ESM.txt", "r")
connections = open("41467_2014_BFncomms5212_MOESM1045_ESM.txt", "r")

disease_data = [[], []] # MeSH Disease Term | PubMed Occurrence
symptom_data = [[], []] # MeSH Symptom Term | PubMed Occurrence
connection_data = [[], [], [], []] # MeSH Symptom Term | MeSH Disease Term | PubMed Occurrence | TFIDF Score

# Read diseases
diseases.readline() # Ignore first line (header)
while True:
	content = diseases.readline()
	if not content:
		break

	content_list = content.split("\t")
	
	# load into disease_data
	disease_data[0].append(content_list[0])

	occurrence = content_list[1].strip() # Strip \n newline + convert into int
	disease_data[1].append(int(occurrence))
diseases.close()

# Read symptoms
symptoms.readline() # Ignore first line (header)
while True:
	content = symptoms.readline()
	if not content:
		break

	content_list = content.split("\t")
	
	# load into symptoms_data
	symptom_data[0].append(content_list[0])

	occurrence = content_list[1].strip() # Strip \n newline + convert into int
	symptom_data[1].append(int(occurrence))
symptoms.close()

# read connections
connections.readline() # Ignore first line (header)
while True:
	content = connections.readline()
	if not content:
		break

	content_list = content.split("\t")
	
	# load into connections data
	connection_data[0].append(content_list[0])
	connection_data[1].append(content_list[1])
	
	occurrence_a = content_list[2].strip()
	connection_data[2].append(int(occurrence_a))

	occurrence_b = content_list[3].strip()
	connection_data[3].append(float(occurrence_b))
connections.close()

In [6]:
### TODO: Add "Unknown Symptom" to match the three datasets --> there are 223 diseases that do not have a listed symptom as per dataset #3 but still have a non-negligible frequency as per Pub Med occurrences

symptom_data[0].append("Unknown Symptom")
symptom_data[1].append(0.000) # Frequency of 0

In [7]:
disease_prob = np.array(disease_data[1]) / sum(disease_data[1])
symptom_prob = np.array(symptom_data[1]) / sum(symptom_data[1])
connection_prob = np.array(connection_data[2]) / sum(connection_data[2])

In [8]:
### TODO: Connect all symptoms to possible diseases (and add idiopathic disease otherwise)

# Create DataFrame
connection_df = pd.DataFrame({"Symptom": connection_data[0], 
							  "Disease": connection_data[1], 
							  "Occurrence": connection_data[2], 
							  "TFIDF Score": connection_data[3]})

# Define nodisease term
nodisease = "Idiopathic Disease"

# Check all diseases in CONNECTIONS corresponds to a disease in DISEASES and vice versa
temp = 0
list_A = []
for var in connection_data[1]:
	if var not in disease_data[0]:
		temp += 1
		list_A.append(var)

temp = 0
list_B = []
for var in disease_data[0]:
	if var not in connection_data[1]:
		temp += 1
		list_B.append(var)

### Observation:	There are 223 diseases in Set #1 that do not have a corresponding symptom as per Set #3
### 				But all diseases listed in Set #3 exist in Set #1 (and therefore have a prior prob.)	

# Check all symptoms in CONNECTIONS corresponds to a symptom in DISEASES and vice versa
temp = 0
list_C = []
for var in connection_data[0]:
	if var not in symptom_data[0]:
		temp += 1
		list_C.append(var)

temp = 0
list_D = []
for var in symptom_data[0]:
	if var not in connection_data[0]:
		temp += 1
		list_D.append(var)

### Observation: EVERY symptom in connections is in symptoms, and vice versa. There are no missing symptoms.

In [9]:
### TODO: Create 322x4442 table to represent symptom x disease co-occurrence

connections_dict = {key: None for key in symptom_data[0]} # Will have 322 + 1 keys, each key-value is list(4442)

# METHOD: Whichever column in the 4442 row-vector a given disease in connections_df for a specific symptom corresponds to, we populate that element with the co-occurrence probability (e.g. the relative probability of that disease occurring relative to other diseases in the set WITHIN an individual symptom). Then, the ENTIRE row-vector is multiplied by the overall symptom's corresponding probability (symptom_prob)
for symptom in symptom_data[0]:
	prob_of_disease = np.array([np.longdouble(0.0)] * len(disease_data[0])) # 4442 long

	filtered = connection_df[connection_df["Symptom"] == symptom]

	# Populate prob_of_disease for that symptom
	for disease in filtered["Disease"]:
		occurrence = filtered[filtered["Disease"] == disease]["Occurrence"] # Get occurrence value
		occurrence = list(occurrence)[0]
		
		index = disease_data[0].index(disease)
		prob_of_disease[index] = np.longdouble(occurrence)
	
	connections_dict[symptom] = prob_of_disease

if True: # Special case for "Unknown Symptom"
	prob_of_disease = np.array([np.longdouble(0.0)] * len(disease_data[0]))

	for disease in list_B: # list_B is where all the diseases w/o associated symptoms are listed
		index = disease_data[0].index(disease)
		prob_of_disease[index] = (disease_data[1])[index] # Just use frequency data for the diseases themselves
	
	connections_dict["Unknown Symptom"] = prob_of_disease	

In [10]:
# TODO: Use connections_dict to define the symptom list for CPD (symptom_cpd_list)

# Create symptom_cpd_list
symptom_cpd_list = [np.longdouble(0.0)] * len(symptom_data[0])

symptom_data[1][-1] = 0.0001 # Set "Unknown Symptom" relative frequency to 0.0001
for i in range(len(symptom_data[0])):
	symptom_cpd_list[i] = connections_dict[symptom_data[0][i]] * symptom_data[1][i] # Set weights via relative symptom frequencies here
symptom_cpd_list_normalized = symptom_cpd_list

# Normalize list so conditional probabilities sum to 1
symptom_list_sum = np.sum(symptom_cpd_list, axis = 0)
symptom_cpd_list_normalized = np.divide(symptom_cpd_list, symptom_list_sum)

# Create Bayesian Network

In [12]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [13]:
bayesianList = [('D', 'S')]

for symptom in symptom_data[0]:
	bayesianList.append(('D', symptom))

model = BayesianNetwork(bayesianList)

In [ ]:
### TODO: "What is the probability that I have body weight problems if I have this disease?"
# Symptom(Yes) | Symptom(No) --> For all 4442 diseases

symptom_prob_list = {key: [([0] * len(disease_data[0])), ([1] * len(disease_data[0]))] for key in symptom_data[0]}

index = 0
for disease in disease_data[0]:
	filtered = connection_df[connection_df["Disease"] == disease]
	for symptom in symptom_data[0]:
		if symptom in list(filtered["Symptom"]):
			yes = filtered[filtered["Symptom"] == symptom]["Occurrence"]
			yes = list(yes)[0]
			all_num = sum(filtered["Occurrence"])

			symptom_prob_list[symptom][0][index] = np.longdouble(yes) / all_num
			symptom_prob_list[symptom][1][index] = np.longdouble(all_num - yes) / all_num

	index += 1


In [ ]:
# Part 2: Define the CPDs (Conditional Probability Distributions)
# See https://pgmpy.org/detailed_notebooks/2.%20Bayesian%20Networks.html for explanation

cpd_disease = TabularCPD(variable = "D", 
						 variable_card = len(disease_prob),
						 values = disease_prob[:, np.newaxis],
						 state_names = {"D": disease_data[0]})
model.add_cpds(cpd_disease)

cpd_symptoms = TabularCPD(variable = "S",
					 	  variable_card = len(symptom_prob),
						  values = np.array(symptom_cpd_list_normalized),
						  evidence = ["D"],
						  evidence_card = [len(disease_prob)],
						  state_names = {"S": symptom_data[0],
					  					 "D": disease_data[0]})
model.add_cpds(cpd_symptoms)

index = 0
for symptom in symptom_data[0]: # Say that the states of a symptom are: "Yes" / "No" -- relative weights of existence relative to every other symptoms
	
	cpd_symptom = TabularCPD(variable = symptom,
							 variable_card = 2,
							 values = symptom_prob_list[symptom],
							 evidence = ["D"],
							 evidence_card = [len(disease_data[0])],
							 state_names = {symptom : ["Yes", "No"],
							 				"D": disease_data[0]})

	model.add_cpds(cpd_symptom)
	
	index += 1

In [ ]:
# Check if the model is valid
assert model.check_model()

# Set Up Interface

In [ ]:

def model_interface():

	print("Welcome to your AI Doctor.")
	while True:
		symptoms_shown = input("Please enter your symptoms using the following format: '[symptom] + [symptom] + ... + [symptom]':")
		symptoms_shown = symptoms_shown.split(" + ") # Ask user to write list of symptoms in format: "[disease] + [disease]..."

		symptom_not_found = []
		for symptom in symptoms_shown:
			if symptom not in symptom_data[0]:
				symptom_not_found.append(symptom)
		
		if len(symptom_not_found) > 0:
			print("")
			print(f"ERROR: We could not find the following symptom(s) in our database: {symptom_not_found}.")
			print(f"Please try again...")
			print("")
		else:
			break
		

	if False: # Test functionality w/ a specific disease's actual list of symptoms
		disease_chosen = "Dysphonia"

		known_symptoms = connection_df[connection_df["Disease"] == disease_chosen]["Symptom"]
		known_symptoms = list(known_symptoms)
		symptoms_shown = known_symptoms

	symptoms_shown = symptoms_shown

	symptoms_list_infer = {key : "Yes" for key in symptoms_shown}

	# Inference
	inference = VariableElimination(model)

	prob_of_disease = inference.query(variables = ["D"], 
									evidence = symptoms_list_infer)

	# Move values from DiscreteFactor to an array of tuples
	disease_array = []
	prob_values = []

	for disease in disease_data[0]:
		disease_array.append([str(disease), np.longdouble(prob_of_disease.get_value(D = disease))])
		prob_values.append(np.longdouble(prob_of_disease.get_value(D = disease)))

	disease_array_sorted = sorted(disease_array, key = lambda l:l[1], reverse = True)

	if False: # testing
		print(disease_array_sorted)
		print(sum(prob_values))

	best_guess = disease_array_sorted[0][0]
	guess_percent = disease_array_sorted[0][1]

	if True: # printing
		print(f"Given your symptoms: {symptoms_shown},")
		print(f"you most likely have the following disease:")
		if (type(best_guess) == str and np.isnan(guess_percent) == False):
			print(f"\t{best_guess} with a {np.round(guess_percent * 100, 2)}% likelihood.")
		else:
			print(f"\tIdiopathic Disease, as we could not identify a diagnosis aligning with your provided symptoms.")
		print("")
		print("If you believe that this diagnosis is incorrect, please enter a different set of symptoms, reduce or increase the set of symptoms you have listed, or see a real doctor.")
		print("NOTE: This program works best for diseases with a >300 PubMed occurrence. If you have a less-common disease or if you enter too few symptoms, you may not receive an accurate result.")

model_interface()